In [ ]:
pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4e5863ab2eb93db11861a9378f35b3b0e95151a8e7009992b46b4056be77620b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.6 MB/s eta 0:00:00
Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.3 MB/s eta 0:00:00
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [ ]:

from datasets import load_dataset

In [ ]:
billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)
billsum_train = billsum["train"].train_test_split(test_size=0.2)  #Further split for training and validation
billsum_test = billsum["test"]

In [ ]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 30652 of the Food and Agricultural Code is amended to read:\n30652.\nAll fees for the issuance of dog license tags and all fines collected pursuant to this division shall be paid into the county, city, or city and county treasury, as the case may be, and shall be used:\n(a) First, to pay fees for the issuance of dog license tags.\n(b) Second, to pay fees, salaries, costs, expenses, or any or all of them for the enforcement of this division and all ordinances which are made pursuant to this division.\n(c) Third, to pay damages to owners of livestock which are killed by dogs.\n(d) Fourth, to pay costs of any hospitalization or emergency care of animals pursuant to Section 597f of the Penal Code.\n(e) Fifth, to pay for initial and in-service training for persons charged with enforcing animal control laws, including animal control officers.\nSEC. 2.\nSection 830.9 of the Penal Code is amended to r

In [ ]:
from transformers import PegasusTokenizerFast

In [ ]:
from transformers import AutoTokenizer
tokenizer = PegasusTokenizerFast.from_pretrained("google/pegasus-cnn_dailymail")
checkpoint="google/pegasus-cnn_dailymail"
print(tokenizer.max_len_single_sentence)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

1023


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1023, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum_train = billsum_train.map(preprocess_function, batched=True)
tokenized_billsum_test = billsum_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/791 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
 #function that passes your predictions and labels to compute to calculate the ROUGE metric
 import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args =  Seq2SeqTrainingArguments(
    output_dir="pegasuscnntest1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_steps=100,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum_train["train"],
    eval_dataset=tokenized_billsum_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.169300,1.864967,0.443500,0.201600,0.301200,0.301100,120.166700
2,1.995100,1.765269,0.456300,0.212100,0.308900,0.308600,123.106100
3,1.892100,1.719462,0.467700,0.217800,0.309400,0.309400,122.454500
4,1.825600,1.695618,0.467900,0.218700,0.312100,0.312000,123.404000
5,1.699700,1.672305,0.474700,0.226700,0.316100,0.316300,123.116200
6,1.711900,1.659469,0.474300,0.224100,0.312900,0.312800,122.333300
7,1.612300,1.647423,0.471200,0.226300,0.314700,0.314900,122.944400
8,1.581400,1.643366,0.476000,0.226300,0.316000,0.316000,122.994900
9,1.634200,1.639986,0.476300,0.226800,0.314300,0.314400,123.540400
10,1.579600,1.638347,0.476200,0.229100,0.315600,0.315800,122.979800


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/tran

TrainOutput(global_step=2640, training_loss=1.7956126155275287, metrics={'train_runtime': 3334.948, 'train_samples_per_second': 2.372, 'train_steps_per_second': 0.792, 'total_flos': 2.283298799291597e+16, 'train_loss': 1.7956126155275287, 'epoch': 10.0})

In [ ]:
evaluation_results = trainer.evaluate(tokenized_billsum_test)

In [ ]:
evaluation_results

{'eval_loss': 1.6742020845413208,
 'eval_rouge1': 0.4812,
 'eval_rouge2': 0.2399,
 'eval_rougeL': 0.3275,
 'eval_rougeLsum': 0.328,
 'eval_gen_len': 123.6613,
 'eval_runtime': 318.3899,
 'eval_samples_per_second': 0.779,
 'eval_steps_per_second': 0.261,
 'epoch': 10.0}